In [1]:
from osgeo import ogr, osr
import requests
import json

# 获取geojson
url = 'https://geo.datav.aliyun.com/areas_v3/bound/410000_full.json'
geojson = requests.get(url)
data = json.loads(geojson.content)

# 准备shp数据源
driver = ogr.GetDriverByName('ESRI Shapefile')
shp_path = 'C:\python爬虫\henan.shp'
data_source = driver.CreateDataSource(shp_path)

# 定义坐标系
srs = osr.SpatialReference()
srs.ImportFromEPSG(4326)

# 创建坐标系
layer = data_source.CreateLayer('province',srs,ogr.wkbMultiPolygon)


feature_def = layer.GetLayerDefn() #获取图层定义
properties = data['features'][0]['properties']
for prop_name in properties.keys():
    field = ogr.FieldDefn(prop_name,ogr.OFTString)
    layer.CreateField(field)
    
    
for feature in data['features']:
    geom = ogr.CreateGeometryFromJson(json.dumps(feature['geometry']))
    shp_feature = ogr.Feature(feature_def) #生成新的特征（Feature），以便将其添加到layer中
    
    for prop_name,prop_value in feature['properties'].items():
        prop_value = str(prop_value) if prop_value is not None else ''
        shp_feature.SetField(prop_name, prop_value)
 
    #创建几何
    shp_feature.SetGeometry(geom)
    #创建图层的特征
    layer.CreateFeature(shp_feature)
    # 销毁要素，释放内存
    shp_feature = None
# 关闭数据源
data_source = None